# Twitter scraper by Paige McKenzie

Includes code to access data used in my [blog post](https://p-mckenzie.github.io/2018/03/12/Bachelor-finale/).

In [ ]:
import twitter
## Uses python-twitter (not twitter)
import pandas as pd
import time
import datetime
import re

### Set up Twitter tokens

In [ ]:
import twitter_tokens

### Start API and pull data

In [ ]:
api = twitter.Api(consumer_key=twitter_tokens.consumer_key,
  consumer_secret=twitter_tokens.consumer_secret,
    access_token_key=twitter_tokens.access_token,
    access_token_secret=twitter_tokens.access_secret)

In [ ]:
print(api.VerifyCredentials().name)

In [ ]:
def add_data(tweets, df):
    try:
        start = max(df.index)
    except:
        start = len(df)
    for idx,line in enumerate(tweets):
        line = twitter.models.Status.AsDict(line)
        s = pd.Series(index=df.columns)
        
        try:
            s['screen_name'] = line['user']['screen_name']
        except:
            s['screen_name'] = ''
        try:
            s['text'] = line['text']
        except:
            s['text'] = ''

        try:
            s['name'] = line['user']['name']
        except KeyError:
            s['name'] = ''

        try:
            s['time'] = line['created_at']
        except:
            s['time'] = 0
        try:
            s['favorites'] = line['favorite_count']
        except KeyError:
            s['favorites'] = 0

        try:
            s['retweets'] = line['retweet_count']
        except KeyError:
            s['retweets'] = 0

        try:
            s['location'] = line['user']['location']
        except KeyError:
            s['location'] = ''

        df.loc[start+idx] = s
        
    for col in df.columns:
        try:
            df[col] = df[col].apply(lambda x:re.sub(r"[^ -~]+", '', x))
        except:
            pass
    return df

In [ ]:
columns = ['screen_name','text', 'name', 'time', 'location', 'favorites', 'retweets']
df = pd.DataFrame(columns=columns)

start = 1
iterations = 0
tweets = []
cont = True

In [ ]:
while cont:
    try:
        new_tweets = api.GetSearch(term='bachelor%20OR%20bachelorette%20OR%20arie%20OR%20becca', 
                                   count=100, max_id=start)
        cont = len(new_tweets)>0
        tweets += new_tweets
        start = min([x.id for x in tweets])
        iterations += 1
    except:
        if len(tweets)>0:
            print 'Adding data at {}. Have scraped {} iterations.'.format(datetime.datetime.now().strftime('%H:%M'), iterations)
            df = add_data(tweets, df)
            tweets = []
            if len(df)>65000:
                print 'Saving data as twitter_{}.csv'.format(iterations)
                df.to_csv('twitter_{}.csv'.format(iterations))
                del df
                df = pd.DataFrame(columns=columns)
        else:
            time.sleep(60*5)
print 'Finished at', datetime.datetime.now().strftime('%H:%M')